In [2]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import KY_utils.KY_baseline_functions as base

# restore saved variables
%store -r summary_property2_ky_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


In [4]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_property_two_year'].values

### Model

In [5]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [40,50]
depth = [5,6]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y,
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.01]
depth = [6]
n_estimators = [200]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_property2_ky_model = {"Logistic": logistic_summary,
                                    "Lasso": lasso_summary,
                                    "LinearSVM": svm_summary,
                                    "RF": rf_summary,
                                    "XGBoost": xgb_summary}
%store summary_property2_ky_model

Stored 'summary_property2_ky_model' (dict)


In [6]:
results = []

for model_name, model_summary in summary_property2_ky_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['best_param'], 
                    model_summary['FL_score']])
results

[['Logistic',
  0.7498865455219538,
  0.005550156843817877,
  {'C': 0.01},
  0.6689841805187504],
 ['Lasso',
  0.7506034313650339,
  0.0038848804841983275,
  {'C': 0.01},
  0.6703541315345699],
 ['LinearSVM',
  0.7528009948587961,
  0.004418821528786809,
  {'C': 0.01},
  0.668910302738296],
 ['RF',
  0.7469065275463455,
  0.022176524158257438,
  {'max_depth': 6, 'n_estimators': 50},
  0.6709194571589177],
 ['XGBoost',
  0.7517690226295729,
  0.02145290842273051,
  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200},
  0.64195936722075]]

In [8]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\baselines\\Two Year\\"
results = [["Property",             
            round(logistic_summary['FL_score'],3), 
            round(lasso_summary['FL_score'], 3), 
            round(svm_summary['FL_score'],3), 
            round(rf_summary['FL_score'],3), 
            round(xgb_summary['FL_score'],3)]]
with open(path + 'Two Year KY_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)